In [1]:
cd ..

/home/amit/Practice/test


In [2]:
#Import standard modules
%run lib/__init__.py
%matplotlib inline

In [3]:
import pymongo

In [4]:
IP='localhost'

In [5]:
client = pymongo.MongoClient(IP, 27017)

Check existing databases

In [6]:
client.database_names()

['local', 'myWiki', 'wikipedia']

Create a reference to a new database for wikipedia information ( lets refer to database refs as _dbr)

In [7]:
myWiki_dbr=client.myWiki

Create a reference to a test collection  ( lets refer to collection refs with _cllr)

In [8]:
test_cllr=myWiki_dbr.test_collection

Add a test record to create db and collection

In [9]:
test_record={ 'message' : 'Test db creation'}
test_cllr.insert_one(test_record)

# Maintenance of DB content 

## Check dbs and collections set up

In [10]:
client.database_names() 

['local', 'myWiki', 'wikipedia']

In [11]:
myWiki_dbr.collection_names()

['pagetestload_collection',
 'test_collection',
 'loads_collection',
 'page_collection',
 'pagetest_collection',
 'category_collection',
 'pageload_collection']

## Clear Collections

## Check documents inserted
- Assign a query to a cursor
- run query by calling cursor
- Use cursor to iterate through query results

In [12]:
cursor=test_cllr.find()
cursor
docs=list(cursor)
docs

[{'_id': ObjectId('5adde80f023fe31e0ff7d85d'), 'message': 'Test db creation'},
 {'_id': ObjectId('5addfe4d023fe324ebb363ed'), 'message': 'Test db creation'},
 {'_id': ObjectId('5ae630a3023fe316b0ce5bcd'), 'message': 'Test db creation'},
 {'_id': ObjectId('5ae75de0023fe3241a990db0'), 'message': 'Test db creation'}]

## Rename a collection

## Check content of collections

In [13]:
cursor=test_cllr.find().count()
cursor

4

In [14]:
loads_cllr=myWiki_dbr.loads_collection
cursor=loads_cllr.find()
cdocs=pd.DataFrame(list(cursor))
try:
    cdocs=cdocs.drop('_id',1)
except:
    cdocs
display(cdocs)

,loaded,master_cat,sub_cats_added
0,Sun Apr 29 21:54:10 2018,Category:Classification algorithms,5
1,Sun Apr 29 22:05:48 2018,Category:business software,226
2,Mon Apr 30 13:57:06 2018,Category:Machine learning,45
3,Mon Apr 30 14:29:48 2018,Category:neymar,0
4,Mon Apr 30 14:30:13 2018,Category:quantum,0
5,Mon Apr 30 14:31:57 2018,Category:business,2215


In [15]:
category_cllr=myWiki_dbr.category_collection
cursor=category_cllr.find().count()
display(cursor)

2491

In [16]:
pagetest_cllr=myWiki_dbr.pagetest_collection
cursor=pagetest_cllr.find().count()
display(cursor)

262

In [17]:
page_cllr=myWiki_dbr.page_collection
cursor=page_cllr.find().count()
display(cursor)

28522

In [18]:
pagetestload_cllr=myWiki_dbr.pagetestload_collection
cursor=pagetestload_cllr.find()
doc=list(cursor)
pd.DataFrame(doc)

,_id,loaded,master_cat,pages_added
0,5ae631a1023fe316d2ab7e8a,Sun Apr 29 21:57:05 2018,Category:Classification algorithms,262


In [19]:
pageload_cllr=myWiki_dbr.pageload_collection
cursor=pageload_cllr.find()
pdocs=pd.DataFrame(list(cursor))
try: 
    pdocs=pdocs.drop('_id',1)
except:
    pdocs
display(pdocs)

,loaded,master_cat,pages_added
0,Sun Apr 29 23:01:39 2018,Category:business software,5124
1,Mon Apr 30 14:09:46 2018,Category:Machine learning,1135
2,Mon Apr 30 14:29:59 2018,Category:neymar,0


In [21]:
def pull_wiki_page(page):
    '''
    pull page extract and category list for a specified page using 
    wikipedia API
    '''
    
    #pull page extract
    params_p = {"action": "query",
            "titles": page,
            "prop": 'extracts' ,
            "format": 'json' ,
        }
            
    response_p = requests.get("https://en.wikipedia.org/w/api.php", params = params_p)
    data = response_p.json()
    
    #pull page categories
    params_c = {"action": "query",
            "titles": page,
            "prop": 'categories' ,
            "format": 'json' ,
            }
            
    response_c = requests.get("https://en.wikipedia.org/w/api.php", params = params_c)
    cats = response_c.json()
            
    return data , cats

In [22]:
import re 
from bs4 import BeautifulSoup
import spacy
import nltk  # natural language toolkit
from nltk.corpus import stopwords

In [23]:
nlp=spacy.load('en')

#download stopwords from nltk
nltk.download('stopwords')

#create list of english stop words from nltk
nltk_stop=stopwords.words('english')
#add addtional items to list of stopwords
nltk_stop.append('displaystyle')

[nltk_data] Downloading package stopwords to /home/amit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
def cleaner(text):
    ''' Clean text data, apply spacy lemmatization and nltk stop words'''
    text = re.sub('<* />','',text)
    text = re.sub('<.*>.*</.*>','', text)
    text = re.sub('[\d]',' ',text)
    text = re.sub('{*}',' ',text)
    text = re.sub('[\n]',' ',text)
    text = re.sub('[^a-zA-Z ]',' ',text)
    text = ' '.join(i.lemma_ for i in nlp(text)
                    if i.orth_ not in nltk_stop)
    text = ' '.join(text.split())
    return text

In [25]:
def read_articles( pageid, page):
        '''
        read page extract and category list from wikipedi 
        API query and add to page dict
        '''
        data, cats=pull_wiki_page(page)
        
        # first data clean - extra from json query and parse html
        article=data['query']['pages'][pageid]['extract']
        soup = BeautifulSoup(article, 'html.parser')
        extract=soup.get_text()
        
        # 2nd data clean including lemmatization and stop words
        
        extract_clean=cleaner(extract)

        return article , soup, extract , extract_clean 

In [26]:
article , soup, extract , extract_clean = read_articles('3771060', 'Accuracy paradox')

In [27]:
extract

'The accuracy paradox for predictive analytics states that predictive models with a given level of accuracy may have greater predictive power than models with higher accuracy. It may be better to avoid the accuracy metric in favor of other metrics such as precision and recall.\nAccuracy is often the starting point for analyzing the quality of a predictive model, as well as an obvious criterion for prediction. Accuracy measures the ratio of correct predictions to the total number of cases evaluated. It may seem obvious that the ratio of correct predictions to cases should be a key metric. A predictive model may have high accuracy, but be useless.\nIn an example predictive model for an insurance fraud application, all cases that are predicted as high-risk by the model will be investigated. To evaluate the performance of the model, the insurance company has created a sample data set of 10,000 claims. All 10,000 cases in the validation sample have been carefully checked and it is known whi

In [39]:
dir(doc)
l=[]
for i in doc:
    if i.lemma_ not in nltk_stop:
        l.append(i.lemma_)

display(len(l))

AttributeError: 'dict' object has no attribute 'lemma_'

In [40]:
nlp(extract)

The accuracy paradox for predictive analytics states that predictive models with a given level of accuracy may have greater predictive power than models with higher accuracy. It may be better to avoid the accuracy metric in favor of other metrics such as precision and recall.
Accuracy is often the starting point for analyzing the quality of a predictive model, as well as an obvious criterion for prediction. Accuracy measures the ratio of correct predictions to the total number of cases evaluated. It may seem obvious that the ratio of correct predictions to cases should be a key metric. A predictive model may have high accuracy, but be useless.
In an example predictive model for an insurance fraud application, all cases that are predicted as high-risk by the model will be investigated. To evaluate the performance of the model, the insurance company has created a sample data set of 10,000 claims. All 10,000 cases in the validation sample have been carefully checked and it is known which 

In [41]:
#look at impact of cleaning
display(len(extract_clean))
extract_clean

2066

'the accuracy paradox predictive analytic state predictive model give level accuracy may great predictive power model high accuracy -PRON- may good avoid accuracy metric favor metric precision recall accuracy often starting point analyze quality predictive model well obvious criterion prediction accuracy measure ratio correct prediction total number case evaluate -PRON- may seem obvious ratio correct prediction case key metric a predictive model may high accuracy useless in example predictive model insurance fraud application case predict high risk model investigate to evaluate performance model insurance company create sample datum set claim all case validation sample carefully check know case fraudulent a table confusion assist analyze quality model the definition accuracy table confusion model m fraud calculation accuracy model m fraud show a m t n t p t n f p f n t p mathrm a m frac tn tp tn fp fn tp tn number true negative case fp number false positive case fn number false negativ

In [42]:
def cleaner_v2(text):
    ''' Clean text data, apply spacy lemmatization and nltk stop words'''
    text = re.sub('{.*}',' ',text)
    #text = re.sub('<* />','',text)   # not neeeded as already removed by beautiful soup
    #text = re.sub('<.*>.*</.*>','', text) # not neeeded as already removed by beautiful soup
    #text = re.sub('[\d]',' ',text) # not needed as removed by last 3 steps
    #text = re.sub('[\n]',' ',text) # not needed as removed by last 3 steps
    text = re.sub('[^a-zA-Z ]',' ',text) # remove numbers and characters not in latin alphabet 
    text = ' '.join(i.lemma_ for i in nlp(text)
                    if i.lemma_ not in nltk_stop)
    text = re.sub('-PRON-',' ',text)  # added by spacy lemmatization ?? - remove
    text = ' '.join(i for i in text.split() if len(i)!=1)  # remove redundant spaces and individual letters
    return text

In [43]:
#look at impact of revised cleaner
ec=cleaner_v2(extract)
print(len(ec))
ec

1870


'accuracy paradox predictive analytic state predictive model give level accuracy may great predictive power model high accuracy may good avoid accuracy metric favor metric precision recall accuracy often starting point analyze quality predictive model well obvious criterion prediction accuracy measure ratio correct prediction total number case evaluate may seem obvious ratio correct prediction case key metric predictive model may high accuracy useless example predictive model insurance fraud application case predict high risk model investigate evaluate performance model insurance company create sample datum set claim case validation sample carefully check know case fraudulent table confusion assist analyze quality model definition accuracy table confusion model fraud calculation accuracy model fraud show tn number true negative case fp number false positive case fn number false negative case tp number true positive case formula definition accuracy table table confusion fraud model frau